In [1]:
import openai
from typing import List
from dotenv import load_dotenv
load_dotenv()

True

## Vanilla OpenAI

In [2]:
client = openai.OpenAI()
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
    ]
)
response.choices[0].message.content   

'Hello! How can I help you today?'

In [3]:
prompt_template = "List down top 10 topics in a bulleted list for a developer to learn on {topic}"
client = openai.OpenAI()

def call_chat_model(messages: List[dict])->str:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.2
    )
    return response.choices[0].message.content

def invoke_chain(topic:str)->str:
    prompt_value = prompt_template.format(topic=topic)
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt_value}
    ]
    return call_chat_model(messages)

invoke_chain(".Net with Azure Services")


'- Introduction to .NET framework\n- C# programming language\n- ASP.NET Core\n- Azure App Services\n- Azure Functions\n- Azure SQL Database\n- Azure Storage\n- Azure DevOps\n- Azure Active Directory\n- Azure Cognitive Services'

## With Langchain

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
prompt = ChatPromptTemplate.from_template(
    "List down top 10 topics in a bulleted list for a developer to learn on {topic}"
)
output_parser = StrOutputParser()
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
chain = ( 
    {'topic':RunnablePassthrough()}
    | prompt
    | model
    | output_parser
    | { 'topics': lambda x: x.split('\n') }
)
chain.invoke("python FastApi")

{'topics': ['- Introduction to FastApi',
  '- Setting up a FastApi project',
  '- Creating API endpoints in FastApi',
  '- Request and response handling in FastApi',
  '- Authentication and authorization in FastApi',
  '- Database integration with FastApi',
  '- Testing FastApi applications',
  '- Deployment of FastApi applications',
  '- Performance optimization in FastApi',
  '- Advanced features and best practices in FastApi']}

## What is this "|"?

In [11]:
class CustomLCEL:
    def __init__(self, value):
        self.value = value
    
    def __or__(self,other):
        if callable(other):
            return CustomLCEL(other(self.value))
        else:
            raise ValueError("Right operand must be a function")
    def result(self):
        return self.value
    
def to_uppercase(s):
    return s.upper()
def to_lowercase(s):
    return s.lower()
def to_title(s):
    return s.title()
def to_capitalize(s):
    return s.capitalize()
def to_swapcase(s):
    return s.swapcase()
def add_exclamation(s):
    return s + "!"
def reverse_string(s):
    return s[::-1]



In [12]:
custom_chain =(
    CustomLCEL("hello world")
    | to_uppercase
    | add_exclamation
    | reverse_string
)
result = custom_chain.result()
print(result)  # Output: "!DLROW OLLEH"

!DLROW OLLEH


## Runnable Interface(Stream, Invoke, Batch)

In [14]:
for s in chain.stream({"topic":"Langchain"}):
    print(s,end="",flush=True)

{'topics': ['- Blockchain technology', '- Smart contracts', '- Cryptography', '- Decentralized applications (dApps)', '- Consensus algorithms', '- Tokenization', '- Distributed ledger technology', '- Solidity programming language', '- Ethereum platform', '- Web3.js integration']}

In [15]:
chain.invoke({"topic":"Entity Framework Core"})

{'topics': ['- Introduction to Entity Framework Core',
  '- Setting up Entity Framework Core in a project',
  '- Creating models and database context',
  '- CRUD operations with Entity Framework Core',
  '- Querying data with LINQ',
  '- Working with migrations',
  '- Understanding relationships in Entity Framework Core',
  '- Using stored procedures with Entity Framework Core',
  '- Performance optimization techniques',
  '- Advanced topics in Entity Framework Core such as transactions and concurrency control']}

In [16]:
chain.batch([{'topic':'.Net Core'},{"topic":"Serilog"}])

[{'topics': ['- Introduction to .Net Core',
   '- Setting up a development environment for .Net Core',
   '- Understanding the architecture of .Net Core',
   '- Building web applications with .Net Core',
   '- Working with databases in .Net Core',
   '- Implementing authentication and authorization in .Net Core',
   '- Testing and debugging in .Net Core',
   '- Deploying .Net Core applications',
   '- Performance optimization in .Net Core',
   '- Continuous integration and deployment with .Net Core']},
 {'topics': ['- Introduction to Serilog',
   '- Setting up Serilog in a .NET application',
   '- Logging levels in Serilog',
   '- Customizing log output in Serilog',
   '- Using sinks in Serilog',
   '- Enriching log data with properties in Serilog',
   '- Structured logging with Serilog',
   '- Filtering log messages in Serilog',
   '- Error handling and exception logging in Serilog',
   '- Best practices for logging with Serilog']}]

In [18]:
# when using Langchain with FastApi, we can usethe Async methods of the above
async for s in chain.astream({"topic":"Dapper"}):
    print(s,end="",flush=True)

{'topics': ['- Introduction to Dapper', '- Installing Dapper', '- Connecting Dapper to a database', '- Performing CRUD operations with Dapper', '- Mapping database tables to C# classes with Dapper', '- Handling transactions with Dapper', '- Using Dapper for complex queries', '- Performance optimization with Dapper', '- Unit testing Dapper code', '- Best practices for using Dapper']}

In [19]:
await chain.ainvoke({"topic":"EFCore"})

{'topics': ['- Introduction to Entity Framework Core',
  '- Setting up EF Core in a project',
  '- Creating models and database context',
  '- CRUD operations with EF Core',
  '- Querying data with EF Core',
  '- Working with migrations in EF Core',
  '- Using LINQ with EF Core',
  '- Handling relationships in EF Core',
  '- Performance optimization in EF Core',
  '- Advanced topics in EF Core such as transactions, stored procedures, and concurrency control']}

In [20]:
await chain.abatch([{'topic':'Docker'},{"topic":"Docker Compose"}])

[{'topics': ['- Introduction to Docker',
   '- Docker installation and setup',
   '- Docker containers',
   '- Docker images',
   '- Docker networking',
   '- Docker volumes',
   '- Docker compose',
   '- Docker swarm',
   '- Docker security',
   '- Docker best practices']},
 {'topics': ['- Introduction to Docker Compose',
   '- Installing Docker Compose',
   '- Creating a Docker Compose file',
   '- Defining services in Docker Compose',
   '- Networking in Docker Compose',
   '- Volumes in Docker Compose',
   '- Environment variables in Docker Compose',
   '- Docker Compose commands',
   '- Docker Compose with multiple environments',
   '- Best practices for using Docker Compose']}]